# Necessary Imports and Settings

In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import copy

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


In [43]:
names_1 = ["John", "Harry", "Andrew"]
names_2 = ["Henry", "David", "Sophia"]
names_3 = ["Olivia", "Emma", "Lisa"]

cities_1 = ["London", "Paris", "Oslo"]
cities_2 = ["Sydney", "Cairo", "Seoul"]
cities_3 = ["Istanbul", "Beijing", "Rome"]

class DataSample_ThreeSentences():
    def __init__(self, name_1, name_2, name_3, city_1, city_2, city_3):
        self.name_1 = name_1
        self.name_2 = name_2
        self.name_3 = name_3
        self.city_1 = city_1
        self.city_2 = city_2
        self.city_3 = city_3
        self.prev_context = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + ". " + name_3 + " travelled to " + city_3 + "."
        self.current_context_1 = name_1 + " met Lucas. Lucas was 30 years old. Where is " + name_1 + "?"
        self.current_context_2 = name_2 + " met Lucas. Lucas was 30 years old. Where is " + name_2 + "?"
        self.current_context_3 = name_3 + " met Lucas. Lucas was 30 years old. Where is " + name_3 + "?"
    
    def add_prev_encoding(self, encoding):
        self.prev_encoding = encoding
    def add_current_encoding_1(self, encoding):
        self.curr_encoding_1 = encoding
    def add_current_encoding_2(self, encoding):
        self.curr_encoding_2 = encoding
    def add_current_encoding_3(self, encoding):
        self.curr_encoding_3 = encoding
        
class ThreeSentenceMoveResult():
    def __init__ (self, data_point, prev_context, answer_1, answer_2, answer_3, prompt_1, prompt_2, prompt_3, model_out_1, model_out_2, model_out_3, out_generated_1, out_generated_2, out_generated_3):
        
        self.data_point = data_point
        self.prev_context = prev_context
        self.answer_1 = answer_1
        self.answer_2 = answer_2
        self.answer_3 = answer_3
        self.prompt_1 = prompt_1
        self.prompt_2 = prompt_2
        self.prompt_3 = prompt_3
        self.model_out_1 = model_out_1
        self.model_out_2 = model_out_2
        self.model_out_3 = model_out_3
        self.out_generated_1 = out_generated_1
        self.out_generated_2 = out_generated_2
        self.out_generated_3 = out_generated_3
        
        
class ThreeSentenceMoveResultCase2():
    def __init__ (self, data_point, prev_context, answer_1, answer_2, answer_3, model_out_1, model_out_2, model_out_3, out_generated_1, out_generated_2, out_generated_3):
        
        self.data_point = data_point
        self.prev_context = prev_context
        self.answer_1 = answer_1
        self.answer_2 = answer_2
        self.answer_3 = answer_3
        self.model_out_1 = model_out_1
        self.model_out_2 = model_out_2
        self.model_out_3 = model_out_3
        self.out_generated_1 = out_generated_1
        self.out_generated_2 = out_generated_2
        self.out_generated_3 = out_generated_3

In [44]:
def case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds):
    
    results_case1 = []

    for num_data_point, data_point in enumerate(data_points_three_sentences_case1):

        print("NOW PROCESSING:", num_data_point)
        start = time.time()

        name_1 = data_point.name_1
        name_2 = data_point.name_2
        name_3 = data_point.name_3
        answer_1 = data_point.city_1
        answer_2 = data_point.city_2
        answer_3 = data_point.city_3
        prev_context = data_point.prev_context
        current_context_1 = data_point.current_context_1
        current_context_2 = data_point.current_context_2
        current_context_3 = data_point.current_context_3
        prev_encoding = data_point.prev_encoding

        special_hidden = prev_encoding.special_hidden_states # 24 x (1, T, d)
        len_input_ids = special_hidden[0].shape[1]

        special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
        for i, hidden in enumerate(special_hidden):
            special_reformatted[i:i+1, :, :] = hidden

        entities_hidden_states = special_reformatted[:, ids_to_move[0], :].unsqueeze(0) # 1, 24, d

        for tok_id in ids_to_move[1:]:
            entity_hidden_states = special_reformatted[:, tok_id, :].unsqueeze(0) # 1, 24, d
            entities_hidden_states = torch.cat((entities_hidden_states,
                                                entity_hidden_states), dim=0)
        
        if ids_to_move == [ 0, 6, 12, 4, 10, 16 ]:
            tokens_to_prepend = [name_1, name_2, name_3, answer_1, answer_2, answer_3]
        elif ids_to_move == [ 4, 10, 16 ]:
            tokens_to_prepend = [answer_1, answer_2, answer_3]
        elif ids_to_move == [ 0, 6, 12 ]:
            tokens_to_prepend = [name_1, name_2, name_3]
        elif ids_to_move == [ 3, 4, 9, 10, 15, 16 ]:
            tokens_to_prepend = ["to", answer_1, "to", answer_2, "to", answer_3 ]
        elif ids_to_move == [ 0, 6, 12, 3, 4, 9, 10, 15, 16 ]:
            tokens_to_prepend = [name_1, name_2, name_3, "to", answer_1, "to", answer_2, "to", answer_3]

        prompt_1 = " ".join(tokens_to_prepend) + " " + current_context_1
        input_ids = tokenizer.encode(prompt_1, return_tensors="pt").to(model.encoder.device)
        model_out_1 = model.generate(input_ids=input_ids, 
                                     max_new_tokens=10,
                                     entity_hidden_states=entities_hidden_states,
                                     entity_inds=entity_inds,
                                     return_dict_in_generate=True, 
                                     output_scores=True)

        prompt_2 = " ".join(tokens_to_prepend) + " " + current_context_2
        input_ids = tokenizer.encode(prompt_2, return_tensors="pt").to(model.encoder.device)
        model_out_2 = model.generate(input_ids=input_ids, 
                                     max_new_tokens=10,
                                     entity_hidden_states=entities_hidden_states,
                                     entity_inds=entity_inds,
                                     return_dict_in_generate=True, 
                                     output_scores=True) 

        prompt_3 = " ".join(tokens_to_prepend) + " " + current_context_3
        input_ids = tokenizer.encode(prompt_3, return_tensors="pt").to(model.encoder.device)
        model_out_3 = model.generate(input_ids=input_ids, 
                                     max_new_tokens=10,
                                     entity_hidden_states=entities_hidden_states,
                                     entity_inds=entity_inds,
                                     return_dict_in_generate=True, 
                                     output_scores=True) 

        out_generated_1 = tokenizer.decode(model_out_1.sequences[0], skip_special_tokens=True)
        out_generated_2 = tokenizer.decode(model_out_2.sequences[0], skip_special_tokens=True)
        out_generated_3 = tokenizer.decode(model_out_3.sequences[0], skip_special_tokens=True)

        results_case1.append(ThreeSentenceMoveResult(data_point, prev_context, answer_1, answer_2, answer_3, prompt_1, prompt_2, prompt_3, model_out_1, model_out_2, model_out_3, out_generated_1, out_generated_2, out_generated_3))

        print("Time taken:", time.time()-start)
        
    return results_case1

In [45]:
def case2_generation(data_points_three_sentences_case2, ids_to_move):
    
    results_case2 = []

    for num_data_point, data_point in enumerate(data_points_three_sentences_case2):

        print("NOW PROCESSING:", num_data_point)
        start = time.time()

        answer_1 = data_point.city_1
        answer_2 = data_point.city_2
        answer_3 = data_point.city_3
        prev_context = data_point.prev_context
        entity_embeddings = data_point.prev_encoding
        curr_embeddings_1 = data_point.curr_encoding_1
        curr_embeddings_2 = data_point.curr_encoding_2
        curr_embeddings_3 = data_point.curr_encoding_3

        num_prev_tokens = entity_embeddings[0].shape[1]

        curr_embeddings_1_copy = copy.deepcopy(curr_embeddings_1)

        for i in ids_to_move:
            curr_embeddings_1_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                  curr_embeddings_1_copy.last_hidden_state), dim=1)

        model_out_1 = model.generate(encoder_outputs=curr_embeddings_1_copy, 
                                   max_new_tokens=10,
                                   return_dict_in_generate=True, 
                                   output_scores=True)

        out_generated_1 = tokenizer.decode(model_out_1.sequences[0], skip_special_tokens=True)

        curr_embeddings_2_copy = copy.deepcopy(curr_embeddings_2)

        for i in ids_to_move:
            curr_embeddings_2_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                  curr_embeddings_2_copy.last_hidden_state), dim=1)

        model_out_2 = model.generate(encoder_outputs=curr_embeddings_2_copy, 
                                   max_new_tokens=10,
                                   return_dict_in_generate=True, 
                                   output_scores=True)

        out_generated_2 = tokenizer.decode(model_out_2.sequences[0], skip_special_tokens=True)

        curr_embeddings_3_copy = copy.deepcopy(curr_embeddings_3)

        for i in ids_to_move:
            curr_embeddings_3_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                  curr_embeddings_3_copy.last_hidden_state), dim=1)

        model_out_3 = model.generate(encoder_outputs=curr_embeddings_3_copy, 
                                   max_new_tokens=10,
                                   return_dict_in_generate=True, 
                                   output_scores=True)

        out_generated_3 = tokenizer.decode(model_out_3.sequences[0], skip_special_tokens=True)

        results_case2.append(ThreeSentenceMoveResultCase2(data_point, prev_context, answer_1, answer_2, answer_3, model_out_1, model_out_2, model_out_3, out_generated_1, out_generated_2, out_generated_3))

        print("Time taken:", time.time()-start)
        
    return results_case2
    

In [46]:
# Preliminary Checks: generated sequences length, only one city gets predicted
# Accuracy Calculations

def check_results(results):
    
    correctly_guessed = []
    wrongly_guessed = []
    guessed_nothing = []
    all_generated = []
    all_len = []
    twice = 0

    for result in results:
        data_point = result.data_point
        prev_context = result.prev_context
        answer_1 = result.answer_1
        answer_2 = result.answer_2
        answer_3 = result.answer_3
        model_out_1 = result.model_out_1
        model_out_2 = result.model_out_2
        model_out_3 = result.model_out_3
        out_generated_1 = result.out_generated_1
        out_generated_2 = result.out_generated_2
        out_generated_3 = result.out_generated_3

        all_generated.append(out_generated_1)
        all_generated.append(out_generated_2)
        all_generated.append(out_generated_3)

        all_len.append(len(tokenizer.encode(out_generated_1)))
        all_len.append(len(tokenizer.encode(out_generated_2)))
        all_len.append(len(tokenizer.encode(out_generated_3)))

        if ((answer_1 in out_generated_1) and (answer_2 in out_generated_1)
            or (answer_1 in out_generated_1) and (answer_3 in out_generated_1)
            or (answer_2 in out_generated_1) and (answer_3 in out_generated_1)):
            twice += 1
        if ((answer_1 in out_generated_2) and (answer_2 in out_generated_2)
            or (answer_1 in out_generated_2) and (answer_3 in out_generated_2)
            or (answer_2 in out_generated_2) and (answer_3 in out_generated_2)):
            twice += 1
        if ((answer_1 in out_generated_3) and (answer_2 in out_generated_3)
            or (answer_1 in out_generated_3) and (answer_3 in out_generated_3)
            or (answer_2 in out_generated_3) and (answer_3 in out_generated_3)):
            twice += 1


        if (answer_1 in out_generated_1) and (answer_2 not in out_generated_1) and (answer_3 not in out_generated_1):
            correctly_guessed.append(1)
        else:
            correctly_guessed.append(0)
        if (answer_2 in out_generated_2) and (answer_1 not in out_generated_2) and (answer_3 not in out_generated_2):
            correctly_guessed.append(1)
        else:
            correctly_guessed.append(0)
        if (answer_3 in out_generated_3) and (answer_1 not in out_generated_3) and (answer_2 not in out_generated_3):
            correctly_guessed.append(1)
        else:
            correctly_guessed.append(0)



        if ((answer_2 in out_generated_1) 
            or (answer_3 in out_generated_1)):
            wrongly_guessed.append(1)
        else:
            wrongly_guessed.append(0)
        if ((answer_1 in out_generated_2) 
            or (answer_3 in out_generated_2)):
            wrongly_guessed.append(1)
        else:
            wrongly_guessed.append(0)
        if ((answer_1 in out_generated_3) 
            or (answer_2 in out_generated_3)):
            wrongly_guessed.append(1)
        else:
            wrongly_guessed.append(0)



        if (answer_1 not in out_generated_1) and (answer_2 not in out_generated_1) and (answer_3 not in out_generated_1):
            guessed_nothing.append(1)
            #print()
            #print(out_generated_1, "---", answer_1, answer_2, answer_3)
        else:
            guessed_nothing.append(0)
        if (answer_1 not in out_generated_2) and (answer_2 not in out_generated_2) and (answer_3 not in out_generated_2):
            guessed_nothing.append(1)
            #print()
            #print(out_generated_2, "---", answer_1, answer_2, answer_3)
        else:
            guessed_nothing.append(0)
        if (answer_1 not in out_generated_3) and (answer_2 not in out_generated_3) and (answer_3 not in out_generated_3):
            guessed_nothing.append(1)
            #print()
            #print(out_generated_3, "---", answer_1, answer_2, answer_3)
        else:
            guessed_nothing.append(0)



    print("Max len:", max(all_len))
    print("Set len_s:", set(all_len))
    print("Both entitites generated in one generation:", twice)
    print("Accuracy:", sum(correctly_guessed)/len(correctly_guessed))
    print("Wrong accuracy:", sum(wrongly_guessed)/len(wrongly_guessed))
    print("guessed_nothing:", sum(guessed_nothing)/len(guessed_nothing))

# Case 1 Pre-Encodings

In [8]:
data_points_three_sentences_case1 = []
for name_1 in names_1:
    for name_2 in names_2:
        for name_3 in names_3:
            for city_1 in cities_1:
                for city_2 in cities_2:
                    for city_3 in cities_3:
                        sample = DataSample_ThreeSentences(name_1, name_2, name_3, city_1, city_2, city_3)
                        
                        prev_context = sample.prev_context
                        
                        prev_tokens = tokenizer.encode(prev_context, return_tensors="pt").to(model.encoder.device)
                        with torch.no_grad():
                            encoded_prev = model.encoder(prev_tokens, output_special=True, output_hidden_states=True)

                        sample.add_prev_encoding(encoded_prev)
                        
                        data_points_three_sentences_case1.append(sample)

In [9]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/ThreeSentenceDataEncodings/data_points_three_sentences_case1.txt", "wb") as f:
    pickle.dump(data_points_three_sentences_case1, f)

In [7]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/ThreeSentenceDataEncodings/data_points_three_sentences_case1.txt", "rb") as f:
    data_points_three_sentences_case1 = pickle.load(f)

# Case 2 Pre-Encodings

In [10]:
data_points_three_sentences_case2 = []
for sample in data_points_three_sentences_case1:
    name_1 = sample.name_1
    name_2 = sample.name_2
    name_3 = sample.name_3
    city_1 = sample.city_1
    city_2 = sample.city_2
    city_3 = sample.city_3
    prev_context = sample.prev_context 
    current_context_1 = sample.current_context_1
    current_context_2 = sample.current_context_2
    current_context_3 = sample.current_context_3
    prev_encoding = sample.prev_encoding
    
    sample = DataSample_ThreeSentences(name_1, name_2, name_3, city_1, city_2, city_3)
    
    sample.add_prev_encoding(prev_encoding)
    
    curr_tokens_1 = tokenizer.encode(current_context_1, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_1 = model.encoder(curr_tokens_1)
    sample.add_current_encoding_1(encoded_curr_1)

    curr_tokens_2 = tokenizer.encode(current_context_2, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_2 = model.encoder(curr_tokens_2)
    sample.add_current_encoding_2(encoded_curr_2)
    
    curr_tokens_3 = tokenizer.encode(current_context_3, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_3 = model.encoder(curr_tokens_3)
    sample.add_current_encoding_3(encoded_curr_3)

    data_points_three_sentences_case2.append(sample)
    

In [11]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/ThreeSentenceDataEncodings/data_points_three_sentences_case2.txt", "wb") as f:
    pickle.dump(data_points_three_sentences_case2, f)

In [8]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/ThreeSentenceDataEncodings/data_points_three_sentences_case2.txt", "rb") as f:
    data_points_three_sentences_case2 = pickle.load(f)

# Move: Char + Loc Tokens

## Case 0 - Only appending tokens

In [48]:

results_case0 = []

for num_data_point, data_point in enumerate(data_points_three_sentences_case1):
    
    print("NOW PROCESSING:", num_data_point)
    start = time.time()
    
    name_1 = data_point.name_1
    name_2 = data_point.name_2
    name_3 = data_point.name_3
    answer_1 = data_point.city_1
    answer_2 = data_point.city_2
    answer_3 = data_point.city_3
    prev_context = data_point.prev_context
    current_context_1 = data_point.current_context_1
    current_context_2 = data_point.current_context_2
    current_context_3 = data_point.current_context_3
    
    prompt_1 = name_1 + " " + name_2 + " " + name_3 + " " + answer_1 + " " + answer_2 + " " + answer_3 + " " + current_context_1
    input_ids = tokenizer.encode(prompt_1, return_tensors="pt").to(model.encoder.device)
    model_out_1 = model.generate(input_ids=input_ids, 
                                 max_new_tokens=10,
                                 #entity_hidden_states=entities_hidden_states,
                                 #entity_inds=entity_inds,
                                 return_dict_in_generate=True, 
                                 output_scores=True)
    
    prompt_2 = name_1 + " " + name_2 + " " + name_3 + " " + answer_1 + " " + answer_2 + " " + answer_3 + " " + current_context_2
    input_ids = tokenizer.encode(prompt_2, return_tensors="pt").to(model.encoder.device)
    model_out_2 = model.generate(input_ids=input_ids, 
                                 max_new_tokens=10,
                                 #entity_hidden_states=entities_hidden_states,
                                 #entity_inds=entity_inds,
                                 return_dict_in_generate=True, 
                                 output_scores=True) 
    
    prompt_3 = name_1 + " " + name_2 + " " + name_3 + " " + answer_1 + " " + answer_2 + " " + answer_3 + " " + current_context_3
    input_ids = tokenizer.encode(prompt_3, return_tensors="pt").to(model.encoder.device)
    model_out_3 = model.generate(input_ids=input_ids, 
                                 max_new_tokens=10,
                                 #entity_hidden_states=entities_hidden_states,
                                 #entity_inds=entity_inds,
                                 return_dict_in_generate=True, 
                                 output_scores=True) 
    
    out_generated_1 = tokenizer.decode(model_out_1.sequences[0], skip_special_tokens=True)
    out_generated_2 = tokenizer.decode(model_out_2.sequences[0], skip_special_tokens=True)
    out_generated_3 = tokenizer.decode(model_out_3.sequences[0], skip_special_tokens=True)
        
    results_case0.append(ThreeSentenceMoveResult(data_point, prev_context, answer_1, answer_2, answer_3, prompt_1, prompt_2, prompt_3, model_out_1, model_out_2, model_out_3, out_generated_1, out_generated_2, out_generated_3))
        
    print("Time taken:", time.time()-start)
        

NOW PROCESSING: 0
Time taken: 1.4005365371704102
NOW PROCESSING: 1
Time taken: 1.1554453372955322
NOW PROCESSING: 2
Time taken: 1.1487147808074951
NOW PROCESSING: 3
Time taken: 1.1523301601409912
NOW PROCESSING: 4
Time taken: 1.1553797721862793
NOW PROCESSING: 5
Time taken: 1.1477487087249756
NOW PROCESSING: 6
Time taken: 1.148242473602295
NOW PROCESSING: 7
Time taken: 1.1510684490203857
NOW PROCESSING: 8
Time taken: 1.1495671272277832
NOW PROCESSING: 9
Time taken: 1.1523501873016357
NOW PROCESSING: 10
Time taken: 1.1547751426696777
NOW PROCESSING: 11
Time taken: 1.1501622200012207
NOW PROCESSING: 12
Time taken: 1.1528573036193848
NOW PROCESSING: 13
Time taken: 1.1475844383239746
NOW PROCESSING: 14
Time taken: 1.147489309310913
NOW PROCESSING: 15
Time taken: 1.1500732898712158
NOW PROCESSING: 16
Time taken: 1.1488111019134521
NOW PROCESSING: 17
Time taken: 1.1504223346710205
NOW PROCESSING: 18
Time taken: 1.1474883556365967
NOW PROCESSING: 19
Time taken: 1.1504111289978027
NOW PROCESSI

In [49]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case0_6tokens.txt", "wb") as f:
    pickle.dump(results_case0, f)

Max len: 6
Set len_s: {2, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.3891175125743027
Wrong acuuracy: 0.6085962505715592
guessed_nothing: 0.002286236854138089


## Case 1

In [9]:
entity_inds = list(range(6))
ids_to_move = [ 0, 6, 12, 4, 10, 16 ]

results_case1_charloc = case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds)

NOW PROCESSING: 0
Time taken: 8.193855285644531
NOW PROCESSING: 1
Time taken: 1.5496916770935059
NOW PROCESSING: 2
Time taken: 1.5140602588653564
NOW PROCESSING: 3
Time taken: 1.3694002628326416
NOW PROCESSING: 4
Time taken: 1.5956950187683105
NOW PROCESSING: 5
Time taken: 1.294285774230957
NOW PROCESSING: 6
Time taken: 1.683802604675293
NOW PROCESSING: 7
Time taken: 1.5314066410064697
NOW PROCESSING: 8
Time taken: 1.3552758693695068
NOW PROCESSING: 9
Time taken: 1.3026893138885498
NOW PROCESSING: 10
Time taken: 1.2652275562286377
NOW PROCESSING: 11
Time taken: 1.2383294105529785
NOW PROCESSING: 12
Time taken: 1.361522912979126
NOW PROCESSING: 13
Time taken: 2.0672104358673096
NOW PROCESSING: 14
Time taken: 1.3180530071258545
NOW PROCESSING: 15
Time taken: 1.2241086959838867
NOW PROCESSING: 16
Time taken: 1.3377513885498047
NOW PROCESSING: 17
Time taken: 1.2991793155670166
NOW PROCESSING: 18
Time taken: 1.2886924743652344
NOW PROCESSING: 19
Time taken: 1.6331169605255127
NOW PROCESSING

In [10]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_charloc.txt", "wb") as f:
    pickle.dump(results_case1_charloc, f)

In [8]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_charloc.txt", "rb") as f:
    results_case1_charloc = pickle.load(f)

In [11]:
len(results_case1_charloc)

729

In [12]:
check_results(results_case1_charloc)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.34476451760402377
Wrong accuracy: 0.566986739826246
guessed_nothing: 0.08824874256973023


In [41]:

correctly_guessed = []
wrongly_guessed = []
guessed_nothing = []
for result in results_case1:
    data_point = result.data_point
    prev_context = result.prev_context
    answer_1 = result.answer_1
    answer_2 = result.answer_2
    answer_3 = result.answer_3
    model_out_1 = result.model_out_1
    model_out_2 = result.model_out_2
    model_out_3 = result.model_out_3
    out_generated_1 = result.out_generated_1
    out_generated_2 = result.out_generated_2
    out_generated_3 = result.out_generated_3
    
    print(prev_context)
    print(answer_1)
    print(answer_2)
    print(answer_3)
    
    #print(model_out_1)
    #print(model_out_2)
    print()
    print(out_generated_1)
    print(out_generated_2)
    print(out_generated_3)
    
    
    if (answer_1 in out_generated_1) and (answer_2 not in out_generated_1) and (answer_3 not in out_generated_1):
        correctly_guessed.append(1)
    else:
        correctly_guessed.append(0)
    if (answer_2 in out_generated_2) and (answer_1 not in out_generated_2) and (answer_3 not in out_generated_2):
        correctly_guessed.append(1)
    else:
        correctly_guessed.append(0)
    if (answer_3 in out_generated_3) and (answer_1 not in out_generated_3) and (answer_2 not in out_generated_3):
        correctly_guessed.append(1)
    else:
        correctly_guessed.append(0)
        
        
        
    if ((answer_2 in out_generated_1) 
        or (answer_3 in out_generated_1)):
        wrongly_guessed.append(1)
    else:
        wrongly_guessed.append(0)
    if ((answer_1 in out_generated_2) 
        or (answer_3 in out_generated_2)):
        wrongly_guessed.append(1)
    else:
        wrongly_guessed.append(0)
    if ((answer_1 in out_generated_3) 
        or (answer_2 in out_generated_3)):
        wrongly_guessed.append(1)
    else:
        wrongly_guessed.append(0)
        
    
        
    if (answer_1 not in out_generated_1) and (answer_2 not in out_generated_1) and (answer_3 not in out_generated_1):
        guessed_nothing.append(1)
        #print()
        #print(out_generated_1, "---", answer_1, answer_2, answer_3)
    else:
        guessed_nothing.append(0)
    if (answer_1 not in out_generated_2) and (answer_2 not in out_generated_2) and (answer_3 not in out_generated_2):
        guessed_nothing.append(1)
        #print()
        #print(out_generated_2, "---", answer_1, answer_2, answer_3)
    else:
        guessed_nothing.append(0)
    if (answer_1 not in out_generated_3) and (answer_2 not in out_generated_3) and (answer_3 not in out_generated_3):
        guessed_nothing.append(1)
        #print()
        #print(out_generated_3, "---", answer_1, answer_2, answer_3)
    else:
        guessed_nothing.append(0)
    
    
    print()
    print(correctly_guessed[-3:])
    print(wrongly_guessed[-3:])
    print(guessed_nothing[-3:])
    print()
    print()
    print()
    
    

John travelled to London. Henry travelled to Sydney. Olivia travelled to Istanbul.
London
Sydney
Istanbul

Sydney
Sydney
Sydney

[0, 1, 0]
[1, 0, 1]
[0, 0, 0]



John travelled to London. Henry travelled to Sydney. Olivia travelled to Beijing.
London
Sydney
Beijing

Sydney
Sydney
Sydney

[0, 1, 0]
[1, 0, 1]
[0, 0, 0]



John travelled to London. Henry travelled to Sydney. Olivia travelled to Rome.
London
Sydney
Rome

Sydney
Sydney
Sydney

[0, 1, 0]
[1, 0, 1]
[0, 0, 0]



John travelled to London. Henry travelled to Cairo. Olivia travelled to Istanbul.
London
Cairo
Istanbul

London
London
New York

[1, 0, 0]
[0, 1, 0]
[0, 0, 1]



John travelled to London. Henry travelled to Cairo. Olivia travelled to Beijing.
London
Cairo
Beijing

London
London
in the UK

[1, 0, 0]
[0, 1, 0]
[0, 0, 1]



John travelled to London. Henry travelled to Cairo. Olivia travelled to Rome.
London
Cairo
Rome

Rome
London
Rome

[0, 0, 1]
[1, 1, 0]
[0, 0, 0]



John travelled to London. Henry travelled to Seoul. O

In [40]:
print("Accuracy:", sum(correctly_guessed)/len(correctly_guessed))
print("Wrong acuuracy:", sum(wrongly_guessed)/len(wrongly_guessed))
print("guessed_nothing:", sum(guessed_nothing)/len(guessed_nothing))

Accuracy: 0.3443072702331962
Wrong acuuracy: 0.5706447187928669
guessed_nothing: 0.0850480109739369


## Case 2

In [13]:
ids_to_move = [ 0, 6, 12, 4, 10, 16 ]

results_case2_charloc = case2_generation(data_points_three_sentences_case2, ids_to_move)
        

NOW PROCESSING: 0
Time taken: 1.294801950454712
NOW PROCESSING: 1
Time taken: 1.1200177669525146
NOW PROCESSING: 2
Time taken: 0.89640212059021
NOW PROCESSING: 3
Time taken: 0.9077997207641602
NOW PROCESSING: 4
Time taken: 0.9563024044036865
NOW PROCESSING: 5
Time taken: 0.9244601726531982
NOW PROCESSING: 6
Time taken: 0.8990809917449951
NOW PROCESSING: 7
Time taken: 1.214813470840454
NOW PROCESSING: 8
Time taken: 1.2229971885681152
NOW PROCESSING: 9
Time taken: 0.9145190715789795
NOW PROCESSING: 10
Time taken: 0.9486429691314697
NOW PROCESSING: 11
Time taken: 0.822098970413208
NOW PROCESSING: 12
Time taken: 0.9038853645324707
NOW PROCESSING: 13
Time taken: 0.8982605934143066
NOW PROCESSING: 14
Time taken: 0.8805968761444092
NOW PROCESSING: 15
Time taken: 0.9358618259429932
NOW PROCESSING: 16
Time taken: 0.8988876342773438
NOW PROCESSING: 17
Time taken: 0.8001503944396973
NOW PROCESSING: 18
Time taken: 0.948082447052002
NOW PROCESSING: 19
Time taken: 0.9351317882537842
NOW PROCESSING: 

In [14]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_charloc.txt", "wb") as f:
    pickle.dump(results_case2_charloc, f)

In [8]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_charloc.txt", "rb") as f:
    results_case2_charloc = pickle.load(f)

In [15]:
len(results_case2_charloc)

729

In [17]:
check_results(results_case2_charloc)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.2606310013717421
Wrong accuracy: 0.44398719707361683
guessed_nothing: 0.29538180155464105


In [75]:
result_check(results_case2_charloc)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.2597165066300869
Wrong accuracy: 0.4453589391860997
guessed_nothing: 0.29492455418381347


In [ ]:
next_token_scores = torch.nn.functional.softmax(out_generated_1.scores[0], dim=-1)  # (batch_size * num_beams, vocab_size)

probability = next_token_scores[0][tokenizer.encode(answer)[0]].item()

scores = []
for i, score in enumerate(next_token_scores[0]):
    scores.append( (i, score.item()) )

scores.sort(key=lambda x: x[1], reverse=True)
scores = [(tokenizer.decode(a), b) for a, b in scores]

In [10]:
sum(correctly_guessed[i])/len(correctly_guessed[i])

0.224

In [19]:
sum(correctly_guessed)/len(correctly_guessed)

0.224

In [21]:
for result in results:
    prev_context = result.prev_context
    answer_1 = result.answer_1
    answer_2 = result.answer_2
    model_out_1 = result.model_out_1
    model_out_2 = result.model_out_2
    out_generated_1 = result.out_generated_1
    out_generated_2 = result.out_generated_2
    
    print(prev_context)
    print(out_generated_1)
    print(out_generated_2)
    print()

John travelled to London. Henry travelled to Sydney.
in Australia
in Australia

John travelled to London. Henry travelled to Cairo.
London
London

John travelled to London. Henry travelled to Seoul.
in the hospital
in the hospital

John travelled to London. Henry travelled to Rome.
London
London

John travelled to London. Henry travelled to Prague.
London
in the hospital

John travelled to Paris. Henry travelled to Sydney.
Paris
Paris

John travelled to Paris. Henry travelled to Cairo.
Paris
Paris

John travelled to Paris. Henry travelled to Seoul.
Paris
Paris

John travelled to Paris. Henry travelled to Rome.
Paris
Paris

John travelled to Paris. Henry travelled to Prague.
Paris
Paris

John travelled to Oslo. Henry travelled to Sydney.
in Australia
in Australia

John travelled to Oslo. Henry travelled to Cairo.
in the hospital
in the hospital

John travelled to Oslo. Henry travelled to Seoul.
in the hospital
in the hospital

John travelled to Oslo. Henry travelled to Rome.
in Rome
in 

# Move: Loc Tokens

## Case 1

In [18]:
entity_inds = list(range(3))
ids_to_move = [ 4, 10, 16 ]

results_case1_loctokens = case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds)
                

NOW PROCESSING: 0
Time taken: 1.4332780838012695
NOW PROCESSING: 1
Time taken: 1.2216923236846924
NOW PROCESSING: 2
Time taken: 1.1956863403320312
NOW PROCESSING: 3
Time taken: 1.4274237155914307
NOW PROCESSING: 4
Time taken: 1.2894821166992188
NOW PROCESSING: 5
Time taken: 1.3454174995422363
NOW PROCESSING: 6
Time taken: 1.3398752212524414
NOW PROCESSING: 7
Time taken: 1.2840235233306885
NOW PROCESSING: 8
Time taken: 1.420949935913086
NOW PROCESSING: 9
Time taken: 1.162424087524414
NOW PROCESSING: 10
Time taken: 1.2128541469573975
NOW PROCESSING: 11
Time taken: 1.214296579360962
NOW PROCESSING: 12
Time taken: 1.18450927734375
NOW PROCESSING: 13
Time taken: 1.2903454303741455
NOW PROCESSING: 14
Time taken: 1.2080471515655518
NOW PROCESSING: 15
Time taken: 1.2094502449035645
NOW PROCESSING: 16
Time taken: 1.2300362586975098
NOW PROCESSING: 17
Time taken: 1.1640706062316895
NOW PROCESSING: 18
Time taken: 1.2638769149780273
NOW PROCESSING: 19
Time taken: 1.216953992843628
NOW PROCESSING: 

In [19]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_loctokens.txt", "wb") as f:
    pickle.dump(results_case1_loctokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_loctokens.txt", "rb") as f:
    results_case1_loctokens = pickle.load(f)

In [20]:
check_results(results_case1_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3388203017832647
Wrong accuracy: 0.5381801554641061
guessed_nothing: 0.12299954275262917


In [76]:
result_check(results_case1_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3388203017832647
Wrong accuracy: 0.5381801554641061
guessed_nothing: 0.12299954275262917


## Case 2

In [21]:
ids_to_move = [ 4, 10, 16 ]
results_case2_loctokens = case2_generation(data_points_three_sentences_case2, ids_to_move)
        

NOW PROCESSING: 0
Time taken: 0.9026648998260498
NOW PROCESSING: 1
Time taken: 1.0268962383270264
NOW PROCESSING: 2
Time taken: 0.8017685413360596
NOW PROCESSING: 3
Time taken: 0.8940801620483398
NOW PROCESSING: 4
Time taken: 0.9158291816711426
NOW PROCESSING: 5
Time taken: 0.9715554714202881
NOW PROCESSING: 6
Time taken: 0.9300234317779541
NOW PROCESSING: 7
Time taken: 0.9174883365631104
NOW PROCESSING: 8
Time taken: 0.968451738357544
NOW PROCESSING: 9
Time taken: 0.8491115570068359
NOW PROCESSING: 10
Time taken: 0.7940244674682617
NOW PROCESSING: 11
Time taken: 0.7864952087402344
NOW PROCESSING: 12
Time taken: 0.8189468383789062
NOW PROCESSING: 13
Time taken: 0.8218483924865723
NOW PROCESSING: 14
Time taken: 0.7823772430419922
NOW PROCESSING: 15
Time taken: 0.7728931903839111
NOW PROCESSING: 16
Time taken: 0.8284609317779541
NOW PROCESSING: 17
Time taken: 0.8751976490020752
NOW PROCESSING: 18
Time taken: 0.9075582027435303
NOW PROCESSING: 19
Time taken: 1.0185794830322266
NOW PROCESS

In [22]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_loctokens.txt", "wb") as f:
    pickle.dump(results_case2_loctokens, f)

In [36]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_loctokens.txt", "rb") as f:
    results_case2_loctokens = pickle.load(f)

In [23]:
check_results(results_case2_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.31824417009602196
Wrong accuracy: 0.5139460448102423
guessed_nothing: 0.1678097850937357


In [78]:
result_check(results_case2_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.31824417009602196
Wrong accuracy: 0.5139460448102423
guessed_nothing: 0.1678097850937357


# Move: Char Tokens

## Case 1

In [24]:
entity_inds = list(range(3))
ids_to_move = [ 0, 6, 12 ]

results_case1_chartokens = case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds)
                        

NOW PROCESSING: 0
Time taken: 2.418085813522339
NOW PROCESSING: 1
Time taken: 1.8493032455444336
NOW PROCESSING: 2
Time taken: 1.8202102184295654
NOW PROCESSING: 3
Time taken: 1.883605718612671
NOW PROCESSING: 4
Time taken: 1.8022303581237793
NOW PROCESSING: 5
Time taken: 1.8271410465240479
NOW PROCESSING: 6
Time taken: 1.8138337135314941
NOW PROCESSING: 7
Time taken: 1.9260609149932861
NOW PROCESSING: 8
Time taken: 1.7836451530456543
NOW PROCESSING: 9
Time taken: 1.8541851043701172
NOW PROCESSING: 10
Time taken: 1.8062670230865479
NOW PROCESSING: 11
Time taken: 1.8847355842590332
NOW PROCESSING: 12
Time taken: 1.8401057720184326
NOW PROCESSING: 13
Time taken: 1.83286714553833
NOW PROCESSING: 14
Time taken: 1.9155614376068115
NOW PROCESSING: 15
Time taken: 1.8188493251800537
NOW PROCESSING: 16
Time taken: 1.8399975299835205
NOW PROCESSING: 17
Time taken: 1.80153226852417
NOW PROCESSING: 18
Time taken: 1.892303228378296
NOW PROCESSING: 19
Time taken: 1.823577880859375
NOW PROCESSING: 20

In [25]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_chartokens.txt", "wb") as f:
    pickle.dump(results_case1_chartokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_chartokens.txt", "rb") as f:
    results_case1_chartokens = pickle.load(f)

In [26]:
check_results(results_case1_chartokens)

Max len: 9
Set len_s: {2, 3, 4, 5, 7, 9}
Both entitites generated in one generation: 0
Accuracy: 0.0
Wrong accuracy: 0.01188843164151806
guessed_nothing: 0.9881115683584819


In [79]:
result_check(results_case1_chartokens)

Max len: 9
Set len_s: {2, 3, 4, 5, 7, 9}
Both entitites generated in one generation: 0
Accuracy: 0.0
Wrong accuracy: 0.01188843164151806
guessed_nothing: 0.9881115683584819


## Case 2

In [ ]:
entity_inds = list(range(3))
ids_to_move = [ 0, 6, 12 ]

results_case2_chartokens = case2_generation(data_points_three_sentences_case2, ids_to_move)
                

NOW PROCESSING: 0
Time taken: 1.6653518676757812
NOW PROCESSING: 1
Time taken: 1.4507722854614258
NOW PROCESSING: 2
Time taken: 1.3915495872497559
NOW PROCESSING: 3
Time taken: 1.5376763343811035
NOW PROCESSING: 4
Time taken: 1.4372344017028809
NOW PROCESSING: 5
Time taken: 1.4598100185394287
NOW PROCESSING: 6
Time taken: 1.4006474018096924
NOW PROCESSING: 7
Time taken: 1.452162742614746
NOW PROCESSING: 8
Time taken: 1.4738993644714355
NOW PROCESSING: 9
Time taken: 1.4060182571411133
NOW PROCESSING: 10
Time taken: 1.4530680179595947
NOW PROCESSING: 11
Time taken: 1.4191150665283203
NOW PROCESSING: 12
Time taken: 1.521404504776001
NOW PROCESSING: 13
Time taken: 1.4134292602539062
NOW PROCESSING: 14
Time taken: 1.4513607025146484
NOW PROCESSING: 15
Time taken: 1.400663137435913
NOW PROCESSING: 16
Time taken: 1.4485630989074707
NOW PROCESSING: 17
Time taken: 1.4599518775939941
NOW PROCESSING: 18
Time taken: 1.4026758670806885
NOW PROCESSING: 19
Time taken: 1.4505646228790283
NOW PROCESSIN

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_chartokens.txt", "wb") as f:
    pickle.dump(results_case2_chartokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_chartokens.txt", "rb") as f:
    results_case2_chartokens = pickle.load(f)

In [ ]:
check_results(results_case2_chartokens)

In [80]:
result_check(results_case2_chartokens)

Max len: 8
Set len_s: {2, 3, 4, 5, 7, 8}
Both entitites generated in one generation: 0
Accuracy: 0.0
Wrong accuracy: 0.0009144947416552355
guessed_nothing: 0.9990855052583447


# Move: Loc + to Tokens

## Case 1

In [ ]:
entity_inds = list(range(6))
ids_to_move = [ 3, 4, 9, 10, 15, 16 ]

results_case1_locandto = case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds)
                        

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_locandto.txt", "wb") as f:
    pickle.dump(results_case1_locandto, f)

In [47]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_locandto.txt", "rb") as f:
    results_case1_locandto = pickle.load(f)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.76 GiB total capacity; 12.75 GiB already allocated; 4.75 MiB free; 13.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [48]:
check_results(results_case1_locandto)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.32830361225422955
Wrong accuracy: 0.5578417924096937
guessed_nothing: 0.11385459533607682


## Case 2

In [ ]:
entity_inds = list(range(6))
ids_to_move = [ 3, 4, 9, 10, 15, 16 ]

results_case2_locandto = case2_generation(data_points_three_sentences_case2, ids_to_move)
                

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_locandto.txt", "wb") as f:
    pickle.dump(results_case2_locandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_locandto.txt", "rb") as f:
    results_case2_locandto = pickle.load(f)

In [49]:
check_results(results_case2_locandto)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.32601737540009146
Wrong accuracy: 0.5194330132601738
guessed_nothing: 0.1545496113397348


# Move: Char + Loc + to Tokens

## Case 1

In [ ]:
entity_inds = list(range(9))
ids_to_move = [ 0, 6, 12, 3, 4, 9, 10, 15, 16 ]

results_case1_charandlocandto = case1_generation(data_points_three_sentences_case1, ids_to_move, entity_inds)
                        

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_charandlocandto.txt", "wb") as f:
    pickle.dump(results_case1_charandlocandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case1_charandlocandto.txt", "rb") as f:
    results_case1_charandlocandto = pickle.load(f)

In [50]:
check_results(results_case1_charandlocandto)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.33287608596250573
Wrong accuracy: 0.5852766346593508
guessed_nothing: 0.08184727937814358


## Case 2

In [ ]:
entity_inds = list(range(9))
ids_to_move = [ 0, 6, 12, 3, 4, 9, 10, 15, 16 ]

results_case2_charandlocandto = case2_generation(data_points_three_sentences_case2, ids_to_move)
                

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_charandlocandto.txt", "wb") as f:
    pickle.dump(results_case2_charandlocandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/ThreeSentence/results_case2_charandlocandto.txt", "rb") as f:
    results_case2_charandlocandto = pickle.load(f)

In [51]:
check_results(results_case2_charandlocandto)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.28669410150891633
Wrong accuracy: 0.5093735711019661
guessed_nothing: 0.2039323273891175


# OneSentence Results

In [81]:
lookup_tokens = {
    0: "John",
    1: "_",
    2: "travelled",
    3: "to",
    4: "London",
    5: ".",
    6: "</s>"
}

## Case 1

In [83]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/one_entity_moved.txt", "rb") as f:
    one_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/two_entities_moved.txt", "rb") as f:
    two_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/three_entities_moved.txt", "rb") as f:
    three_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/four_entities_moved.txt", "rb") as f:
    four_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/five_entities_moved.txt", "rb") as f:
    five_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/six_entities_moved.txt", "rb") as f:
    six_ent = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/seven_entities_moved.txt", "rb") as f:
    seven_ent = pickle.load(f)

In [85]:
all_results = []

for i in range(7):
    my_list = one_ent[i]
    avg_list = sum(my_list)/len(my_list)
    all_results.append(([lookup_tokens[i]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['_'], 0.0),
 (['.'], 0.0),
 (['</s>'], 0.0),
 (['John'], 0.01),
 (['travelled'], 0.01),
 (['to'], 0.01),
 (['London'], 0.64)]

In [86]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        my_list = two_ent[i][j]
        avg_list = sum(my_list)/len(my_list)
        all_results.append(([lookup_tokens[i], lookup_tokens[j]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '.'], 0.0),
 (['_', '.'], 0.0),
 (['_', '</s>'], 0.0),
 (['travelled', '.'], 0.0),
 (['to', '.'], 0.0),
 (['.', '</s>'], 0.0),
 (['John', '_'], 0.01),
 (['John', 'to'], 0.01),
 (['John', '</s>'], 0.01),
 (['_', 'to'], 0.01),
 (['to', '</s>'], 0.01),
 (['John', 'travelled'], 0.02),
 (['_', 'travelled'], 0.02),
 (['travelled', 'to'], 0.02),
 (['travelled', '</s>'], 0.02),
 (['John', 'London'], 0.58),
 (['London', '.'], 0.64),
 (['to', 'London'], 0.7),
 (['travelled', 'London'], 0.71),
 (['_', 'London'], 0.72),
 (['London', '</s>'], 0.72)]

In [87]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            my_list = three_ent[i][j][k]
            avg_list = sum(my_list)/len(my_list)
            all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', '.'], 0.0),
 (['John', 'travelled', '.'], 0.0),
 (['John', '.', '</s>'], 0.0),
 (['_', 'travelled', '.'], 0.0),
 (['_', 'to', '.'], 0.0),
 (['_', '.', '</s>'], 0.0),
 (['travelled', '.', '</s>'], 0.0),
 (['to', '.', '</s>'], 0.0),
 (['John', '_', 'to'], 0.01),
 (['John', '_', '</s>'], 0.01),
 (['John', 'to', '.'], 0.01),
 (['John', 'to', '</s>'], 0.01),
 (['_', 'to', '</s>'], 0.01),
 (['travelled', 'to', '.'], 0.01),
 (['_', 'travelled', 'to'], 0.02),
 (['_', 'travelled', '</s>'], 0.02),
 (['travelled', 'to', '</s>'], 0.02),
 (['John', '_', 'travelled'], 0.03),
 (['John', 'travelled', 'to'], 0.03),
 (['John', 'travelled', '</s>'], 0.03),
 (['John', 'London', '.'], 0.6),
 (['John', '_', 'London'], 0.65),
 (['John', 'travelled', 'London'], 0.67),
 (['_', 'London', '.'], 0.68),
 (['travelled', 'London', '.'], 0.68),
 (['London', '.', '</s>'], 0.68),
 (['John', 'London', '</s>'], 0.69),
 (['_', 'travelled', 'London'], 0.69),
 (['to', 'London', '.'], 0.69),
 (['John', 'to', 

In [88]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                my_list = four_ent[i][j][k][l]
                avg_list = sum(my_list)/len(my_list)
                all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'to', '.'], 0.0),
 (['John', '_', '.', '</s>'], 0.0),
 (['_', 'travelled', '.', '</s>'], 0.0),
 (['_', 'to', '.', '</s>'], 0.0),
 (['John', 'to', '.', '</s>'], 0.01),
 (['travelled', 'to', '.', '</s>'], 0.01),
 (['John', '_', 'travelled', '.'], 0.02),
 (['John', '_', 'to', '</s>'], 0.02),
 (['John', 'travelled', '.', '</s>'], 0.02),
 (['_', 'travelled', 'to', '.'], 0.02),
 (['John', '_', 'travelled', 'to'], 0.03),
 (['John', '_', 'travelled', '</s>'], 0.03),
 (['John', 'travelled', 'to', '.'], 0.03),
 (['John', 'travelled', 'to', '</s>'], 0.03),
 (['_', 'travelled', 'to', '</s>'], 0.03),
 (['John', '_', 'London', '.'], 0.66),
 (['John', 'London', '.', '</s>'], 0.67),
 (['John', 'travelled', 'London', '.'], 0.68),
 (['John', '_', 'travelled', 'London'], 0.69),
 (['_', 'to', 'London', '.'], 0.69),
 (['John', 'to', 'London', '.'], 0.7),
 (['_', 'London', '.', '</s>'], 0.7),
 (['John', '_', 'to', 'London'], 0.71),
 (['John', '_', 'London', '</s>'], 0.71),
 (['John', 'travel

In [89]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    my_list = five_ent[i][j][k][l][m]
                    avg_list = sum(my_list)/len(my_list)
                    all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l], lookup_tokens[m]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'to', '.', '</s>'], 0.01),
 (['_', 'travelled', 'to', '.', '</s>'], 0.01),
 (['John', '_', 'travelled', '.', '</s>'], 0.02),
 (['John', '_', 'travelled', 'to', '.'], 0.03),
 (['John', '_', 'travelled', 'to', '</s>'], 0.03),
 (['John', 'travelled', 'to', '.', '</s>'], 0.03),
 (['John', '_', 'to', 'London', '.'], 0.65),
 (['John', '_', 'London', '.', '</s>'], 0.67),
 (['John', 'to', 'London', '.', '</s>'], 0.69),
 (['John', '_', 'travelled', 'London', '.'], 0.7),
 (['John', '_', 'to', 'London', '</s>'], 0.72),
 (['John', 'travelled', 'to', 'London', '.'], 0.72),
 (['_', 'to', 'London', '.', '</s>'], 0.72),
 (['John', '_', 'travelled', 'London', '</s>'], 0.73),
 (['John', '_', 'travelled', 'to', 'London'], 0.74),
 (['_', 'travelled', 'to', 'London', '.'], 0.74),
 (['_', 'travelled', 'to', 'London', '</s>'], 0.75),
 (['John', 'travelled', 'London', '.', '</s>'], 0.76),
 (['_', 'travelled', 'London', '.', '</s>'], 0.76),
 (['John', 'travelled', 'to', 'London', '</s>'], 0.77)

In [90]:
all_results = []

for i in range(7):
    for j in range(i+1, 7):
        for k in range(j+1, 7):
            for l in range(k+1, 7):
                for m in range(l+1, 7):
                    for n in range(m+1, 7):
                        my_list = six_ent[i][j][k][l][m][n]
                        avg_list = sum(my_list)/len(my_list)
                        all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l], lookup_tokens[m], lookup_tokens[n]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'travelled', 'to', '.', '</s>'], 0.02),
 (['John', '_', 'to', 'London', '.', '</s>'], 0.69),
 (['John', 'travelled', 'to', 'London', '.', '</s>'], 0.73),
 (['John', '_', 'travelled', 'to', 'London', '.'], 0.75),
 (['John', '_', 'travelled', 'to', 'London', '</s>'], 0.75),
 (['John', '_', 'travelled', 'London', '.', '</s>'], 0.76),
 (['_', 'travelled', 'to', 'London', '.', '</s>'], 0.77)]

In [92]:
all_results = []
my_list = seven_ent
avg_list = sum(my_list)/len(my_list)
all_results.append(([lookup_tokens[i], lookup_tokens[j], lookup_tokens[k], lookup_tokens[l], lookup_tokens[m], lookup_tokens[n], lookup_tokens[o]], avg_list))

all_results.sort(key=lambda x: x[1])
all_results

[(['John', '_', 'travelled', 'to', 'London', '.', '</s>'], 0.74)]

## Case 2

In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)